In [ ]:
# An example where the polyharmonic spline is the solution

using PyPlot

ENV["MPLBACKEND"]="tkagg" # if you need

x, y = randn(500), randn(500)
z = exp.(-(x.^2 .+ y.^2))
S2 = PolyharmonicSpline(2,[x y],z)

n = 20
xgrid = ones(n) * range(-3, stop = 3, length = n)'
ygrid = range(-3, stop = 3, length = n) * ones(n)'

xx = reshape(xgrid, n * n)
yy = reshape(ygrid, n * n)

zz = interpolate(S2, xx, yy)
zgrid = reshape(zz, n, n);

plot_surface(xgrid, ygrid, zgrid, alpha = 0.5)
scatter3D(x, y, z, color = "r")
show()

# Visualize the sparsity of the 2D Laplacian and Matern matrices

# Take the laplacian matrix and plot it as a heatmap
# Implement Dirichlet, Neumann, and Periodic BCs.

L = 1:300
K = 1:300
xmax = 300
ymax = 300
length_array = length(L)*length(K)
control_indices = 1:15:length_array
number_of_control_points = length(control_indices)
weights = ones(number_of_control_points)
control_values = zeros(number_of_control_points)
xcoordinate = zeros(number_of_control_points)
ycoordinate = zeros(number_of_control_points)
stdx = 50
stdy = 50
mux = 150
muy = 150
for i = 1:number_of_control_points
    xcoor = mod(control_indices[i], xmax)
    ycoor = floor(control_indices[i]/xmax) - 1
    control_values[i] = exp(-((xcoor - mux)/stdx)^2 - ((ycoor - muy)/stdy)^2)
    xcoordinate[i] = xcoor
    ycoordinate[i] = ycoor
end
scatter3D(xcoordinate,ycoordinate,control_values,color="m")


function evaluate_rbfkernel(r)
    if (r <= 1e-13)
        return 0.0
    else
        return r*r*log(r)*0.217147241
    end
end

# This cell runs slowly. 
fvalues = zeros(length_array)
count = 1

for i = 1:ymax
    for j = 1:xmax
        xcoor = j-1
        ycoor = i-1
        for k = 1:number_of_control_points
            r = sqrt((xcoordinate[k] - xcoor)^2 +  (ycoordinate[k] - ycoor)^2)
            fvalues[count] = fvalues[count] + weights[k]*evaluate_rbfkernel(r/100)
        end
        count += 1
    end
end
        

n = 300
xgrid = ones(n) * range(1, stop = 300, length = n)'
ygrid = range(1, stop = 300, length = n) * ones(n)'
fvalues_mat = reshape(fvalues, xmax, ymax)
plot_surface(xgrid, ygrid, fvalues_mat, alpha = 0.5)

heatmap(fvalues_mat, c = :balance)

restored_img1, punched_img = Matern2D(300, 300, fvalues_mat[1:300, 1:300], 0, [(100, 100)], 50);

heatmap(restored_img1, c = :balance)

plot1 = heatmap(fvalues_mat, c = :balance, title="RBF Kernel Data", pointsize=15)
plot2 = heatmap(punched_img, c = :balance, title="Data with missing points")
plot3 = heatmap(restored_img1, c = :balance, title="Matern Interpolated Image")
plot4 = heatmap(abs.(restored_img1 .- fvalues_mat), c = :balance, title = "Error in Mat Interp")
Plots.plot(plot1, plot2, plot3, plot4, layout = (2, 2), legend = false)
# png("RBF_Errors_Matern.png")

# Verifying PolyharmonicSpline code

xarray = Float64[]
yarray = Float64[]
zarray = Float64[]
for x in 1:xmax
    for y in 1:ymax
        if((x -100)^2 + (y-100)^2 > 50^2)
            xarray = append!(xarray, x);
            yarray = append!(yarray, y);
            zarray = append!(zarray, fvalues_mat[y,x]);
        end
    end
end

S2 = PolyharmonicSpline(2,[xarray yarray],zarray);

zz = interpolate(S2,xgrid,ygrid)
zz_reshape = reshape(zz, xmax,ymax)

# 3D Example

function evaluate_rbfkernel3D(r)
    return r
end


L = 1:50
K = 1:50
H = 1:50
length_array = length(L)*length(K)*length(H)
control_indices = 1:15:length_array
number_of_control_points = length(control_indices)
weights = ones(number_of_control_points)
control_values = zeros(number_of_control_points)
xcoordinate = zeros(number_of_control_points)
ycoordinate = zeros(number_of_control_points)
zcoordinate = zeros(number_of_control_points)

stdx = 8
stdy = 8
mux = 25
muy = 25
for i = 1:number_of_control_points
    xcoor = mod(control_indices[i], xmax)
    ycoor = floor(control_indices[i]/xmax) - 1
    control_values[i] = exp(-((xcoor - mux)/stdx)^2 - ((ycoor - muy)/stdy)^2)
    xcoordinate[i] = xcoor
    ycoordinate[i] = ycoor
end
scatter3D(xcoordinate,ycoordinate,control_values,color="m")

